<a href="https://colab.research.google.com/github/404isabel/NLP/blob/master/IsabelVazquez_Problem_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk

In [2]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
#The input text, that will be used to make the predictions
input_text="Just A Rather Very Intelligent System a.k.a JARVIS is created by Tony Stark natural-language and a sophisticated artificial intelligence user interface computer system, named after Edwin Jarvis, the butler who worked for Howard Stark. Though its primary duty is to automate Stark’s Malibu estate, the lifelike program fulfills many other needs for Stark, like being an information source for him, a diagnostic tool, a consultant and a voice of reason in Stark’s life. It was also responsible to provide security for Tony Stark's Mansion and Stark Tower. After creating the Mark II armor, Stark uploaded JARVIS into all of the Iron Man Armors, as well as allowing him to interact with the other Avengers, giving them valuable information during combat. JARVIS may be the one intellect Stark feels most comfortable opening up to. JARVIS can object to Stark’s commands if necessary. JARVIS speaks with a refined British accent, and is capable of back talk, sarcasm and condescension. During the Ultron Offensive, JARVIS was destroyed by Ultron, although his remaining programming codes unknowingly continued to thwart Ultron's plans of gaining access to nuclear missiles. His remains were found by Stark, who uploaded them into a synthetic body made of vibranium and, in conjunction with Ultron's personality and an Infinity Stone. JARVIS' duties were then taken over by FRIDAY. "

In [0]:
#Replace invalid characters
input_text=input_text.replace("’","'")
input_text=input_text.replace(",","")

Tags all the words of the input text

In [9]:
#Categorize all the words in nouns, prepositions, verbs, etc..
from nltk import TreebankWordTokenizer
tbt = TreebankWordTokenizer()
tags = nltk.pos_tag(tbt.tokenize(input_text))
print(tags)
print(len(tags))

[('Just', 'RB'), ('A', 'DT'), ('Rather', 'RB'), ('Very', 'NNP'), ('Intelligent', 'NNP'), ('System', 'NNP'), ('a.k.a', 'JJ'), ('JARVIS', 'NNP'), ('is', 'VBZ'), ('created', 'VBN'), ('by', 'IN'), ('Tony', 'NNP'), ('Stark', 'NNP'), ('natural-language', 'NN'), ('and', 'CC'), ('a', 'DT'), ('sophisticated', 'JJ'), ('artificial', 'JJ'), ('intelligence', 'NN'), ('user', 'NN'), ('interface', 'NN'), ('computer', 'NN'), ('system', 'NN'), ('named', 'VBN'), ('after', 'IN'), ('Edwin', 'NNP'), ('Jarvis', 'NNP'), ('the', 'DT'), ('butler', 'NN'), ('who', 'WP'), ('worked', 'VBD'), ('for', 'IN'), ('Howard', 'NNP'), ('Stark.', 'NNP'), ('Though', 'NNP'), ('its', 'PRP$'), ('primary', 'JJ'), ('duty', 'NN'), ('is', 'VBZ'), ('to', 'TO'), ('automate', 'VB'), ('Stark', 'NNP'), ("'s", 'POS'), ('Malibu', 'NNP'), ('estate', 'NN'), ('the', 'DT'), ('lifelike', 'NN'), ('program', 'NN'), ('fulfills', 'NNS'), ('many', 'JJ'), ('other', 'JJ'), ('needs', 'NNS'), ('for', 'IN'), ('Stark', 'NNP'), ('like', 'IN'), ('being', 'VB

Tokenize the input text

In [10]:
#Tokenize the input text
tokens=tbt.tokenize(input_text) 
print(len(tokens))

225


Function to get the next Linguistic Units

In [0]:
#This function gets the type of the word, based on the tags
def getType(i):  
  tag=tags[i]
  wordType=tag[1] #NNP, NN, JJ, etc...
  return wordType

#This function gets the next Linguistic Unit (not only the word)
#Params:
#tokens: bigram or trigram
#i: index of the element in the input text
#n: 2 (bigram), 3(trigram)
def getNextLinguisticUnits(tokens,i,n):
  words=[]
  
  #The first word
  word=tokens[i+n]
  
  #Detects the type of word 
  wordType=getType(i+n) 
  pos=i+n+1
  linguisticUnit=word
  
  #This code creates and adds the linguistic units given a bigram or trigram, depending on the type of the word
  
  #Prepositions, determiners, modal auxiliaries, to, posesives (') --> this words are not usually a linguistic unit by themselves
  #This code generates a string "prep" that will be added later to the next words
  prep=""
  if wordType in ["IN", "DT","MD","TO","POS"]: 
    prep=linguisticUnit
    #Search the next word
    go=True        
    while wordType in ["IN", "DT","MD","TO","POS"] and pos<len(tokens) and go:
      w=tokens[pos]      
      wordType=getType(pos)
      if wordType in ["IN", "DT","MD","TO","POS"]:
        prep+=" "+w               
        pos=pos+1

      if w[len(w)-1:len(w)]=='.':go=False #If the word has a dot the function doesn't continue searching the next word

    prep+=" "        
  
  if prep!="":
    linguisticUnit=tokens[pos]
    wordType=getType(pos)

  #JJ: Adjetives
  #Adjetives can be followed by other adjetives or nouns.
  #In this code, the prepositions are added to the adjetives (if there was any), and the adjetives are added toghether
  if wordType == 'JJ': 
    if prep!="":
      pos=pos+1
    go=True
    while wordType == 'JJ' and pos+1<len(tokens)-1 and go:
      adj=tokens[pos]      
      wordType=getType(pos)
      
      if wordType == 'JJ':
        linguisticUnit+=" "+adj 
        pos+=1
      if adj[len(adj)-1:len(adj)]=='.':go=False
    
    w=tokens[pos]      
    wordType=getType(pos)
    
    if wordType in ["NN","NNS"] and w[len(w)-1:len(w)]!='.':
        linguisticUnit+=" "+w

    words.append(prep+linguisticUnit.replace(".","")) 

  #NNP: Proper nouns
  #In this code, the prepositions are added to the proper nouns (if there was any), and the proper nouns are added toghether: Example: Tony Stark
  elif wordType == 'NNP':  
    if prep!="":
      pos=pos+1
    go=True 
    while wordType == 'NNP' and pos<len(tokens)-1 and go:
      w=tokens[pos]        
      wordType=getType(pos)
            
      if wordType == 'NNP':
        linguisticUnit+=" "+w                 
        pos=pos+1
      if w[len(w)-1:len(w)]=='.':go=False        
    words.append(prep+linguisticUnit.replace(".",""))

  #NN, NNS: Nouns
  #In this code, the prepositions are added to the nouns (if there was any), and the nouns are added toghether
  elif wordType in ["NN","NNS"]:       
    if linguisticUnit[:len(linguisticUnit)-1]!='.':
      words.append(prep+linguisticUnit.replace(".",""))
      if prep!="":
        pos=pos+1 
      go=True
      while wordType in ["NN","NNS"] and pos<len(tokens)-1 and go:
        w=tokens[pos]
        wordType=getType(pos)                  
        if wordType in ["NN","NNS"]:
          linguisticUnit+=" "+w
          words.append(prep+linguisticUnit.replace(".",""))          
          pos=pos+1
        if w[len(w)-1:len(w)]=='.':go=False 
  
  #The rest of the other words  
  else:
    #It's a linguistic unit   
    words.append(prep+(linguisticUnit).replace(".",""))
   

  return words

Function to create a n-gram (bigram or trigram)

In [0]:
#Creates a n-gram
#params
#tokens: tokens of words
#n: n=3 (trigram) or n=2 (bigram)
def createNgram(tokens,n):
  ngrams = {}
  for i in range(len(tokens)-n):
      k=()
      if n==3: 
        k = (tokens[i],tokens[i+1],tokens[i+2])          
      elif n==2:
        k = (tokens[i],tokens[i+1])
      if k not in ngrams: #if the bigram doesn't exists, it is created
          ngrams[k] = []
      
      words=getNextLinguisticUnits(tokens,i,n) #get the next 'words'   
      for word in words: ngrams[k].append(word) #append the words to the n-gram                                    
  return ngrams

Test the function createNgram

In [126]:
#Test the function createNgram with a trigram
trigram=createNgram(tokens,3)
print(trigram)

{('Just', 'A', 'Rather'): ['Very Intelligent System'], ('A', 'Rather', 'Very'): ['Intelligent System'], ('Rather', 'Very', 'Intelligent'): ['System'], ('Very', 'Intelligent', 'System'): ['aka'], ('Intelligent', 'System', 'a.k.a'): ['JARVIS'], ('System', 'a.k.a', 'JARVIS'): ['is'], ('a.k.a', 'JARVIS', 'is'): ['created'], ('JARVIS', 'is', 'created'): ['by Tony Stark'], ('is', 'created', 'by'): ['Tony Stark'], ('created', 'by', 'Tony'): ['Stark'], ('by', 'Tony', 'Stark'): ['natural-language'], ('Tony', 'Stark', 'natural-language'): ['and'], ('Stark', 'natural-language', 'and'): ['a sophisticated artificial intelligence'], ('natural-language', 'and', 'a'): ['sophisticated artificial intelligence'], ('and', 'a', 'sophisticated'): ['artificial intelligence'], ('a', 'sophisticated', 'artificial'): ['intelligence', 'intelligence user', 'intelligence user interface', 'intelligence user interface computer', 'intelligence user interface computer system'], ('sophisticated', 'artificial', 'intellig

In [127]:
#Test the function createNgram with a bigram
bigram=createNgram(tokens,2)
print(bigram)

{('Just', 'A'): ['Rather'], ('A', 'Rather'): ['Very Intelligent System'], ('Rather', 'Very'): ['Intelligent System'], ('Very', 'Intelligent'): ['System'], ('Intelligent', 'System'): ['aka'], ('System', 'a.k.a'): ['JARVIS'], ('a.k.a', 'JARVIS'): ['is'], ('JARVIS', 'is'): ['created'], ('is', 'created'): ['by Tony Stark'], ('created', 'by'): ['Tony Stark'], ('by', 'Tony'): ['Stark'], ('Tony', 'Stark'): ['natural-language', "'s Mansion"], ('Stark', 'natural-language'): ['and'], ('natural-language', 'and'): ['a sophisticated artificial intelligence'], ('and', 'a'): ['sophisticated artificial intelligence', 'voice'], ('a', 'sophisticated'): ['artificial intelligence'], ('sophisticated', 'artificial'): ['intelligence', 'intelligence user', 'intelligence user interface', 'intelligence user interface computer', 'intelligence user interface computer system'], ('artificial', 'intelligence'): ['user', 'user interface', 'user interface computer', 'user interface computer system'], ('intelligence', 

Function to predict the next word

In [0]:
#Function used to predict the next word
#params:
#input: input text
#output: word 'linguistic unit' predicted
def predictNextWord(input): 
  if len(input)<2:
    print("At least two words are required")
  else:
    inputTokens=tbt.tokenize(input)
    #if the sentence has only two words
    if len(inputTokens)==2:
      ngrams=createNgram(tokens,2) #Creates de bi-gram
      lastTokens=inputTokens[len(inputTokens)-2:len(inputTokens)] 
      k = (lastTokens[0],lastTokens[1])  
      try:          
         return ngrams[k]
      except:
         return [] #Nothing could be predicted
    else:
      ngrams=createNgram(tokens,3) #Creates the tri-gram
      lastTokens=inputTokens[len(inputTokens)-3:len(inputTokens)] 
      k = (lastTokens[0],lastTokens[1],lastTokens[2])      
      try:      
        return ngrams[k]
      except:
        #if nothing could be predicted, it tries with a bi-gram
        ngrams=createNgram(tokens,2)
        k = (lastTokens[1],lastTokens[2])        
        try:
          return ngrams[k]
        except:
          return [] #Nothing could be predicted
    

### Predict the next 'word' given an input text
**Inputs:**

JARVIS was destroyed

JARVIS is created by

JARVIS can object to Stark's

a sophisticated artificial

**Output:**

All the possible outputs given the input



In [129]:
input1="JARVIS was destroyed"
input2="JARVIS is created by"
input3="JARVIS can object to Stark's"
input4="a sophisticated artificial"
print(input1+" --> "+str(predictNextWord(input1)))
print(input2+" --> "+str(predictNextWord(input2)))
print(input3+" --> "+str(predictNextWord(input3)))
print(input4+" --> "+str(predictNextWord(input4)))


JARVIS was destroyed --> ['by Ultron']
JARVIS is created by --> ['Tony Stark']
JARVIS can object to Stark's --> ['commands']
a sophisticated artificial --> ['intelligence', 'intelligence user', 'intelligence user interface', 'intelligence user interface computer', 'intelligence user interface computer system']


Examples with other inputs

In [133]:
#Examples with other inputs
input4="the butler who worked"
print(input4+" --> "+str(predictNextWord(input4)))
input5="like being an information"
print(input5+" --> "+str(predictNextWord(input5)))
input6="Mansion and Stark Tower" 
print(input6+" --> "+str(predictNextWord(input6))) #Nothing could be predicted, as ther is a dot after the sentence in the text.
input7="information during"
print(input7+" --> "+str(predictNextWord(input7)))
input8="JARVIS speaks"
print(input8+" --> "+str(predictNextWord(input8))) #Nothing similar was found in the text, given "in a boat" or "a boat"
input9="I was in a boat"
print(input9+" --> "+str(predictNextWord(input9)))
input10="who uploaded them"
print(input10+" --> "+str(predictNextWord(input10)))

the butler who worked --> ['for Howard Stark']
like being an information --> ['source']
Mansion and Stark Tower --> []
information during --> ['combat']
JARVIS speaks --> ['with a refined British accent']
I was in a boat --> []
who uploaded them --> ['into a synthetic body']


Calculate the probabilities for each trigram

In [134]:
#Calculate the probabilities for each trigram
ngram_probabilities =  {}
for k,words in trigram.items():
      d = {}
      n = 0
      for w in words:
          if w not in d:
              d[w] = 0
          d[w] +=1
          n +=1
      for w,c in d.items():
          d[w] = float(c)/n
      ngram_probabilities[k] = d
print(ngram_probabilities)

{('Just', 'A', 'Rather'): {'Very Intelligent System': 1.0}, ('A', 'Rather', 'Very'): {'Intelligent System': 1.0}, ('Rather', 'Very', 'Intelligent'): {'System': 1.0}, ('Very', 'Intelligent', 'System'): {'aka': 1.0}, ('Intelligent', 'System', 'a.k.a'): {'JARVIS': 1.0}, ('System', 'a.k.a', 'JARVIS'): {'is': 1.0}, ('a.k.a', 'JARVIS', 'is'): {'created': 1.0}, ('JARVIS', 'is', 'created'): {'by Tony Stark': 1.0}, ('is', 'created', 'by'): {'Tony Stark': 1.0}, ('created', 'by', 'Tony'): {'Stark': 1.0}, ('by', 'Tony', 'Stark'): {'natural-language': 1.0}, ('Tony', 'Stark', 'natural-language'): {'and': 1.0}, ('Stark', 'natural-language', 'and'): {'a sophisticated artificial intelligence': 1.0}, ('natural-language', 'and', 'a'): {'sophisticated artificial intelligence': 1.0}, ('and', 'a', 'sophisticated'): {'artificial intelligence': 1.0}, ('a', 'sophisticated', 'artificial'): {'intelligence': 0.2, 'intelligence user': 0.2, 'intelligence user interface': 0.2, 'intelligence user interface computer':